In [1]:
import pandas as pd
from pathlib import Path

In [4]:
DATA_DIR = Path("data/processing_checkpoint")
path_clean_dataset = DATA_DIR / "01_cleaned_train.parquet"

In [5]:
df_clean_churn = pd.read_parquet(path_clean_dataset)

In [7]:
df_clean_churn.sample(5)

,gender,level,userId,page,sessionId,itemInSession,length,song,artist,time,registration,metropolitan_area,region,operating_system,browser
2739034,M,paid,1632245,NextSong,147879,256,157.85751,The End Of Medicine,The New Pornographers,2018-11-02 05:07:53,2018-08-24 03:02:18,Baltimore-Columbia-Towson,MD,iPad,Safari
14732470,M,paid,1741473,NextSong,18825,69,280.13669,Du bist nicht allein,WESTERNHAGEN (HCL),2018-10-27 00:01:02,2018-09-06 08:08:06,Denver-Aurora-Lakewood,CO,Linux,Firefox
16282969,F,paid,1798783,NextSong,17709,6,215.92771,Beggin',Madcon,2018-10-20 00:22:52,2018-09-25 12:36:17,Prescott,AZ,Windows,Chrome
3409111,F,paid,1774647,NextSong,57498,92,143.59465,The Movement Of The Nile,Kotipelto,2018-10-10 16:32:54,2018-09-26 08:38:19,Green Bay,WI,Windows,Chrome
16737117,M,paid,1776162,NextSong,26385,55,327.05261,What's Your Name,Future Loop Foundation,2018-11-02 03:25:25,2018-09-21 09:35:29,Washington-Arlington-Alexandria,DC-VA-MD-WV,Macintosh,Safari


In [ ]:
df_users_base = df_clean_churn[["userId", "gender", "registration", "operating_system", "browser"]].drop_duplicates(subset=["userId"])

In [10]:
len(df_users_base)

19140

In [11]:
df_users_base = df_users_base.set_index("userId")